## SRPT vs. BASRPT

In [1]:
%load_ext autoreload
import trafpy
import trafpy.generator as tpg

import os
import pickle
import matplotlib.pyplot as plt

path_to_benchmark_data = os.path.dirname(trafpy.__file__)+'/../data/testbed_data/ndf50_1s_university_testbed_data_v3.obj'
filehandler = open(path_to_benchmark_data, 'rb')
tb_dict = pickle.load(filehandler)

print(tb_dict.keys())

dict_keys(['benchmark_data', 'benchmarks', 'envs', 'config'])


In [2]:
%autoreload
envs = tb_dict['envs']
print(envs)

[<trafpy.manager.src.simulators.dcn.DCN object at 0x7ff1085b07c0>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff017992880>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff0143e4820>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff01310a6d0>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff011d71b20>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff01037dca0>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff00d617940>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff00b3a6580>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff00927b640>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff0073680d0>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff0040c8bb0>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff001883820>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7ff00df01190>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7feff9868190>, <trafpy.manager.src.simulators.dcn.DCN object at 0x7feff70e2550>, <trafpy.m

## TODO

### Key Stats to Compare
- Average flow completion time
- 99th percentile flow completion time
- Queue length
- Throughput

### Key Plots

In [ ]:
%autoreload
for env in envs:
    print('Net capacity: {}'.format(env.network.graph['max_nw_capacity']))
    print(env.get_scheduling_session_summary(print_summary=True))

In [ ]:
%autoreload
# collect testbed stats into dict
testbed_stats = {'test_{}'.format(i): 
                 {envs[i].scheduler.scheduler_name: 
                  {'load_rate': envs[i].load, 
                   'load_frac': envs[i].load/envs[i].network.graph['max_nw_capacity'],
                   'avrg_fct': envs[i].avrg_fct,
                   'nn_fct': envs[i].nn_fct,
                   'q_dict': envs[i].queue_evolution_dict,
                   'throughput': envs[i].throughput}
                 } for i in range(len(envs))}

# find classes
classes = []
for test in testbed_stats.keys():
    for _class in testbed_stats[test].keys():
        if _class not in classes:
            classes.append(_class)

In [ ]:
%autoreload
# avrg fct
plot_dict = {_class: {'x_values': [], 'y_values': []} for _class in classes}

for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    plot_dict[_class]['x_values'].append(testbed_stats[test][_class]['load_frac'])
    plot_dict[_class]['y_values'].append(testbed_stats[test][_class]['avrg_fct'])

fig = tpg.plot_val_scatter(plot_dict=plot_dict,xlabel='Load', ylabel='Average FCT (us)', show_fig=True)

In [ ]:
%autoreload
# 99th percentile fct
plot_dict = {_class: {'x_values': [], 'y_values': []} for _class in classes}

for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    plot_dict[_class]['x_values'].append(testbed_stats[test][_class]['load_frac'])
    plot_dict[_class]['y_values'].append(testbed_stats[test][_class]['nn_fct'])

fig = tpg.plot_val_scatter(plot_dict=plot_dict,xlabel='Load', ylabel='99th Percentile FCT (us)', show_fig=True)

In [ ]:
%autoreload
import numpy

# queue length
plot_dict = {}
for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    load = testbed_stats[test][_class]['load_frac']
    try:
        plot_dict[load][_class]['x_values'] = None
        plot_dict[load][_class]['y_values'] = None
    except KeyError:
        # not yet added this load
        try:
            plot_dict[load][_class] = {}
            plot_dict[load][_class]['x_values'] = None
            plot_dict[load][_class]['y_values'] = None
        except KeyError:
            plot_dict[load] = {}
            plot_dict[load][_class] = {}
            plot_dict[load][_class]['x_values'] = None
            plot_dict[load][_class]['y_values'] = None

src = 'server_2'
dst='server_10'
for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    load = testbed_stats[test][_class]['load_frac']
    plot_dict[load][_class]['x_values'] = testbed_stats[test][_class]['q_dict'][src][dst]['times'] 
    plot_dict[load][_class]['y_values'] = testbed_stats[test][_class]['q_dict'][src][dst]['queue_lengths']
    
for load in plot_dict.keys():
    tpg.plot_val_line(plot_dict=plot_dict[load],xlabel='Time (us)', ylabel='{}-{} Queue Length (Bytes)'.format(src,dst), show_fig=True)

In [ ]:
%autoreload
# throughput
plot_dict = {}
for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    load = testbed_stats[test][_class]['load_frac']
    try:
        plot_dict[load]['x_values'].append(_class[0])
        plot_dict[load]['y_values'].append(testbed_stats[test][_class]['throughput'])
    except KeyError:
        try:
            plot_dict[load] = {}
            plot_dict[load]['x_values'].append(_class[0])
            plot_dict[load]['y_values'].append(testbed_stats[test][_class]['throughput'])
        except KeyError:
            plot_dict[load] = {}
            plot_dict[load]['x_values'] = []
            plot_dict[load]['y_values'] = []
            plot_dict[load]['x_values'].append(_class[0])
            plot_dict[load]['y_values'].append(testbed_stats[test][_class]['throughput'])
    
print(plot_dict)
for load in plot_dict.keys():
    tpg.plot_val_bar(x_values=plot_dict[load]['x_values'],y_values=plot_dict[load]['y_values'],xlabel='Scheduler', ylabel='Load {} Throughput (B/s)'.format(str(round(load,2))), show_fig=True)

## Tuning BASRPT

In [ ]:
%autoreload
import trafpy
import trafpy.generator as tpg

import os
import pickle
import matplotlib.pyplot as plt

path_to_benchmark_data = os.path.dirname(trafpy.__file__)+'/../data/testbed_data/ndf10_university_testbed_data_basrpt_tuning.obj'
filehandler = open(path_to_benchmark_data, 'rb')
tb_dict = pickle.load(filehandler)

print(tb_dict.keys())




In [ ]:
envs = tb_dict['envs']
print(envs)

## TODO

### Key Stats to Compare
- Throughput vs. V
- Average FCT vs. V
- 99th Percentile vs. V
- Queue Evolution (V composite)

### Key Plots

In [ ]:
for env in envs:
    print('Net capacity: {}'.format(env.network.graph['max_nw_capacity']))
    print(env.get_scheduling_session_summary(print_summary=True))

In [ ]:
# collect testbed stats into dict
testbed_stats = {'test_{}'.format(i): 
                 {envs[i].scheduler.scheduler_name: 
                  {'load_rate': envs[i].load, 
                   'load_frac': envs[i].load/envs[i].network.graph['max_nw_capacity'],
                   'avrg_fct': envs[i].avrg_fct,
                   'nn_fct': envs[i].nn_fct,
                   'q_dict': envs[i].queue_evolution_dict,
                   'throughput': envs[i].throughput,
                   'V': envs[i].scheduler.V}
                 } for i in range(len(envs))}

# find classes
classes = []
for test in testbed_stats.keys():
    for _class in testbed_stats[test].keys():
        if _class not in classes:
            classes.append(_class)

In [ ]:
# avrg fct
plot_dict = {_class: {'x_values': [], 'y_values': []} for _class in classes}

for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    plot_dict[_class]['x_values'].append(testbed_stats[test][_class]['V'])
    plot_dict[_class]['y_values'].append(testbed_stats[test][_class]['avrg_fct'])

fig = tpg.plot_val_scatter(plot_dict=plot_dict,xlabel='V', ylabel='Average FCT (us)', show_fig=True)

In [ ]:
# 99th percentile fct
plot_dict = {_class: {'x_values': [], 'y_values': []} for _class in classes}

for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    plot_dict[_class]['x_values'].append(testbed_stats[test][_class]['V'])
    plot_dict[_class]['y_values'].append(testbed_stats[test][_class]['nn_fct'])

fig = tpg.plot_val_scatter(plot_dict=plot_dict,xlabel='V', ylabel='99th Percentile FCT (us)', show_fig=True)

In [ ]:
# throughput
plot_dict = {_class: {'x_values': [], 'y_values': []} for _class in classes}

for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    plot_dict[_class]['x_values'].append(testbed_stats[test][_class]['V'])
    plot_dict[_class]['y_values'].append(testbed_stats[test][_class]['throughput'])

fig = tpg.plot_val_scatter(plot_dict=plot_dict,xlabel='V', ylabel='Throughput (B/s)', show_fig=True)

In [ ]:
import numpy

# queue length
plot_dict = {}
for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    load = testbed_stats[test][_class]['load_frac']
    try:
        plot_dict[load][_class]['x_values'] = None
        plot_dict[load][_class]['y_values'] = None
    except KeyError:
        # not yet added this load
        try:
            plot_dict[load][_class] = {}
            plot_dict[load][_class]['x_values'] = None
            plot_dict[load][_class]['y_values'] = None
        except KeyError:
            plot_dict[load] = {}
            plot_dict[load][_class] = {}
            plot_dict[load][_class]['x_values'] = None
            plot_dict[load][_class]['y_values'] = None

src = 'server_2'
dst='server_10'
for test in testbed_stats.keys():
    _class = list(testbed_stats[test].keys())[0]
    load = testbed_stats[test][_class]['load_frac']
    plot_dict[load][_class]['x_values'] = testbed_stats[test][_class]['q_dict'][src][dst]['times'] 
    plot_dict[load][_class]['y_values'] = testbed_stats[test][_class]['q_dict'][src][dst]['queue_lengths']
    
for load in plot_dict.keys():
    tpg.plot_val_line(plot_dict=plot_dict[load],xlabel='Time (us)', ylabel='{}-{} Queue Length (Bytes)'.format(src,dst), show_fig=True)

In [8]:
%autoreload
import trafpy
from trafpy.manager import EnvAnalyser

analyser = EnvAnalyser(env=envs[0])

analyser.compute_flow_summary()
print(analyser.measurement_duration)

8337581.9310495285
